In [1]:
import arcgis as GIS
import arcpy.mp
import os
import arcpy
import string, random

## New Functions AUGUST 2021
listWorkSpaces(workPath), listGDBitems(GDBpath), deleteFeatureClass(GDBpath,fcName), newFCfromSelect(GDBpath, selectFrom, selector, selType)

In [2]:
# Return list of workspaces in specified arcpy.env.workspace path
def listWorkSpaces(workPath):
    try:
        arcpy.env.workspace = workPath
        wsList = []
        ws = arcpy.ListWorkspaces("*", "FileGDB")
        for i,w in enumerate(ws):
            wsList.append(w)
            print(f'{i} - {w}')
        return wsList
    except:
        display(f'ERROR listing WorkSpaces in {workPath}')
        
# function to list all Feature Classes in specified FileGeoDatabase
def listItems(GDBpath):
    try:
        arcpy.env.workspace = GDBpath
        fcList = []
        fc = arcpy.ListFeatureClasses()
        print(f'Feature Classes/Shapefile in {GDBpath}: ')
        for i,f in enumerate(fc):
            fcList.append(f)
            print(f'{i} - {f}')
        return fcList
    except:
        display(f'ERROR listing Feature Classes in {GDBpath}')

# Delete feature class given path to GDB and fc name
def deleteFeatureClass(GDBpath,fcName):
    try:
        quest = (input(f'DELETE FEATURE CLASS - {GDBpath}\\{fcName} ??? - ') or 'NO').upper()
        if quest == 'YES':
            result = arcpy.Delete_management(f'{GDBpath}\\{fcName}')
            display(f'File: {fcName} delete = {result}')
        else:
            display(f'Did not delete: {fcName}')
    except:
        display(f'ERROR DELETING {fcName}')


# Make Selecttion and Create New Feature Class into function
def newFCfromSelect(GDBpath, selectFrom, selector, selType):
    try:
        switchSel = (input(f'Switch the resulting selection(y/N)? ')).upper()
        # set workspace GDB, overwrite = True
        arcpy.env.workspace,arcpy.env.overwriteOutput = GDBpath, True
        arcpy.Compact_management(GDBpath)
        # create random string to append to file name, prevents duplication errors
        rand = ''.join(random.choices(string.ascii_uppercase + string.digits, k = 8))
        # tempLayer name from selectFrom name and random 8 character string
        # clear any previous selection and create temporary layer
        arcpy.SelectLayerByAttribute_management(selectFrom, "CLEAR_SELECTION")
        arcpy.MakeFeatureLayer_management(selectFrom, 'tempLayer') 
        # clear selection on selector and perfrom on temporary layer
        arcpy.SelectLayerByAttribute_management('tempLayer', "CLEAR_SELECTION")
        arcpy.SelectLayerByLocation_management('tempLayer', selType, selector)
        if switchSel == 'Y':
            arcpy.SelectLayerByAttribute_management('tempLayer', "SWITCH_SELECTION")
        # Check to see if any features were selected
        matchcount = int(arcpy.GetCount_management('tempLayer')[0]) 
        if matchcount == 0:
            print('no features matched spatial and attribute criteria')
        else:
            arcpy.CopyFeatures_management('tempLayer', (f'{selectFrom}_{rand}_{selType}'))
            display(f'{matchcount} Features selected and written to: {selectFrom}_{rand}_{selType}')
    except:
        display(f'ERROR Creating Selection: {selectFrom} {selType} {selector}')
        
def setCRS(envPath):
    try:
        test, count = 1, 0
        while test != 0:    
            dirList,layList = [], []
            arcpy.env.workspace = envPath
            arcpy.env.overwriteOutput = True
            print(f'ArcPy Workspace: {arcpy.env.workspace}')
            os.chdir(arcpy.env.workspace)
            print(f'Subdirectories')
            for i, item in enumerate(os.scandir(envPath)):
                if item.is_dir():
                    dirList.append(item.path)
                    print(f'{i} - {item.path}')
            ind = int(input(f'Change to Sub-Directory?<ENTER for none> ') or 999)
            if ind != 999:
                arcpy.env.workspace = dirList[ind]
                print(f'ArcPy Workspace: {arcpy.env.workspace}')
            for i, item in enumerate(arcpy.ListFeatureClasses()):
                layList.append(item)
                print(f'{i} - {item}')
            ind = int(input(f'TARGET LAYER TO DEFINE PROJECTION: ') or 999)
            if ind != 999:
                CRS = arcpy.SpatialReference(4326)
                arcpy.DefineProjection_management(layList[ind],CRS)
                print(f'Successfully Defined Projection for {layList[ind]}')
                count += 1
            test = int(input(f'PRESS 1 TO CONTINUE? ') or 0)
        print(f'Set CRS for {count} layers')
    except:
        print(f'ERROR Defining Projection')

## ArcPro Mapping Functions 2021


In [54]:
def setDir():
    curDir = os.path.abspath(os.getcwd())
    listDir = os.listdir()
    print(f'Current Directory: {curDir}')
    print(f'Directory Contents:')
    for i, item in enumerate(listDir):
        print(f'{i} - {item}')
    ind = int(input(f'Change to Sub-Directory?<ENTER for none> ') or 999)
    if ind != 999:
        os.chdir(f'{curDir}\\{listDir[ind]}')
        curDir = os.path.abspath(os.getcwd())
        listDir = os.listdir()
        print(f'Current Directory: {curDir}')
        print(f'Directory Contents:')
        for i, item in enumerate(listDir):
            print(f'{i} - {item}')
    pathName = input(f'Enter Path to change Directory?<ENTER for none> ') or 'NO'
    if pathName != 'NO':
        os.chdir(pathName)
        curDir = os.path.abspath(os.getcwd())
    return curDir

def loadProj():
    try:
        curDir = os.path.abspath(os.getcwd())
        listDir = os.listdir()
        print(f'ArcPro Projects in Directory: ')
        for i, item in enumerate(listDir):
            print(f'{i} - {item}')
        ind = int(input(f'Project Index Number? ') or 999)
        aprx = arcpy.mp.ArcGISProject(f'{curDir}\\{listDir[ind]}')
        print(f'PROJECT LOADED: {listDir[ind]} - {aprx.filePath}')
        return aprx
    except:
        print(f'COULD NOT FIND check for .aprx project')

def getMap(aprx):
    maps = aprx.listMaps('*')
    for i, item in enumerate(maps):
        print(f'{i} - {item.name}')
    ind = int(input(f'Choose Map Index? ') or 999)
    display(f'Active Map is: {maps[ind].name}')
    return maps[ind]

def listLayers(m):
    try:    
        lyrs = m.listLayers()
        for i, item in enumerate(lyrs):
            print(f'{i} - {item.name} - vis: {item.visible}')
        return lyrs
    except: 
        print(f'FAILED to List Layers for {m}')

def getLayer(m):
    try:
        lyrs = m.listLayers()
        for i, item in enumerate(lyrs):
            print(f'{i} - {item.name} - vis: {item.visible}')
        ind = int(input(f'Choose Layer Index? ') or 999)
        display(f'Layer: {lyrs[ind].name}')
        return lyrs[ind]
    except:
        print(f'FAILED to get Layers for {m}')
        
def layersFields(lyrs):
    try:
        for lyr in lyrs:
            print(f'{lyr.name}')
            if lyr.isFeatureLayer == True:
                fields = arcpy.ListFields(lyr)
                for field in fields:
                    print(f'NAME: {field.name}')
    except:
        print(f'Could not list fields for layerList: {lyrs}')

def mergeLayers(lyrs, targetField, outputName):
    try:
        mergeList, rand = [], ''.join(random.choices(string.ascii_uppercase + string.digits, k = 8))
        for lyr in lyrs:
            if lyr.isFeatureLayer == True:
                fields = arcpy.ListFields(lyr)
                for field in fields:
                    if field.name == targetField:
                        print(f'MATCH: {field.name} - {lyr.name} - {lyr.dataSource}')
                        mergeList.append(lyr)
        arcpy.Merge_management(mergeList, (f'{os.path.abspath(os.getcwd())}\\{outputName}-{rand}.shp'), '', '')
        print(f'Successful Merge - Output: {os.path.abspath(os.getcwd())}\\{outputName}-{rand}.shp')
        return mergeList
    except:
        print(f'Could not merge layers for layerList: {lyrs}')
        
def getLayout(aprx):
    lyt = aprx.listLayouts()
    for i, item in enumerate(lyt):
        print(f'{i} - {item.name}')
    ind = int(input(f'Choose Layout Index? ') or 999)
    display(f'Layer: {lyt[ind].name}')
    return lyt[ind]

def getElement(lyt):
    el = lyt.listElements()
    for i, item in enumerate(el):
        print(f'{i} - {item.name}')
    ind = int(input(f'Choose Element Index? ') or 999)
    display(f'Layer: {el[ind].name}')
    return el[ind]

def addLyr(m):
    #try:
        lyrDir = 'Y:\\LAYERS\\'
        lyrs = os.listdir(lyrDir)
        for i, item in enumerate(lyrs):
            print(f'{i} - {item}')
        ind = int(input(f'Choose Layer Index? ') or 999)
        #dirPath = 'Y:\\LAYERS\\'
        layerPath = (f'{lyrDir}{lyrs[ind]}')
        lyr = arcpy.mp.LayerFile(layerPath)
        addLyr = m.addLayer(lyr, 'TOP')
        print(f'{lyr} added to {m.name}')
        
def importMxd(aprx):
    try:
        setDir()
        listDir = os.listdir()
        print(f'Directory Contents:')
        for i, item in enumerate(listDir):
            print(f'{i} - {item}')
        ind = int(input(f'Choose .mxd to Import into Project ') or 999)
        if int != 999:
            curDir = os.path.abspath(os.getcwd())
            mxd = listDir[ind]
        aprx.importDocument(f'{curDir}\\{mxd}')
        print(f'Successfully imported {curDir}\\{mxd}')
        return aprx
    except:
        print(f'Could not import .mxd')

def delMulLayers():
    try:
        curDir, delList = os.path.abspath(os.getcwd()), []
        print(f'Current Directory: {curDir}')
        pathName = input(f'Enter Path to change Directory?<ENTER for none> ') or 'NO'
        if pathName != 'NO':
            os.chdir(pathName)
            curDir = os.path.abspath(os.getcwd())
        listDir = os.listdir()
        print(f'Current Directory: {curDir}')
        print(f'Directory Contents:')
        for i, item in enumerate(listDir):
            files = item.split('.')
            if ((files[1].upper() == 'SHP') and (len(files) < 3)):
                print(f'{i} - {item} - {files[1].upper()}')
                delList.append(item)
        for item in delList:
            delQuest = input(f'DELETE - {item} <YES/NO>') or 'NO'
            if delQuest.upper() == 'YES':
                deleteFeatureClass(curDir, item)
        return curDir
    except:
        print(f'Could not perform delete in {os.path.abspath(os.getcwd())}')

In [ ]:
work = listWorkSpaces('Y:\\DATA_2017\\RICE_DATA_2021\\')

In [ ]:
arcpy.env.workspace = 'Y:\\DATA_2017\\RICE_DATA_2021\\RICE_DATA_2016\\'
arcpy.env.workspace

In [ ]:
# add layers to mxd function
# arcpy.mapping does not work with new arcpy for ArcGIS PRO!!!

#mxd = arcpy.mapping.MapDocument(f'Y:\PROJECTS\MNOMEN_2018\RICE-2021\STEVE RICE MAP 2021.mxd')
#df = arcpy.mapping.ListDataFrames(mxd, "New Data Frame")[0]
#addLayer = arcpy.mapping.Layer(r"C:\Project\Data\Orthophoto.lyr")
#arcpy.mapping.AddLayer(df, addLayer, "BOTTOM")
#mxd.saveACopy(r"C:\Project\Project2.mxd")
#del mxd, addLayer

In [46]:
# Blank .aprx location: Y:\PROJ-2021\TEMPLATES\BLANK-PROJ
# change to directory with blank twmplate
curDir = setDir()
# load blank .aprx project
aprx = loadProj()

Current Directory: Y:\PROJECTS\MNOMEN_2018\RICE-2021
Directory Contents:
0 - RICE-2016-ONLY.mxd
1 - STEVE RICE MAP 2021 - Shortcut.lnk
2 - STEVE RICE MAP 2021.mxd
Change to Sub-Directory?<ENTER for none> 
Enter Path to change Directory?<ENTER for none> Y:\PROJ-2021\TEMPLATES\BLANK-PROJ
ArcPro Projects in Directory: 
0 - BLANK-PROJ.aprx
1 - BLANK-PROJ.gdb
2 - BLANK-PROJ.tbx
3 - ImportLog
4 - Index
Project Index Number? 0
PROJECT LOADED: BLANK-PROJ.aprx - Y:\PROJ-2021\TEMPLATES\BLANK-PROJ\BLANK-PROJ.aprx


In [47]:
# Rice .mxd dir: Y:\PROJECTS\MNOMEN_2018\RICE-2021\
aprx2 = importMxd(aprx)

Current Directory: Y:\PROJ-2021\TEMPLATES\BLANK-PROJ
Directory Contents:
0 - BLANK-PROJ.aprx
1 - BLANK-PROJ.gdb
2 - BLANK-PROJ.tbx
3 - ImportLog
4 - Index
Change to Sub-Directory?<ENTER for none> 
Enter Path to change Directory?<ENTER for none> Y:\PROJECTS\MNOMEN_2018\RICE-2021\
Directory Contents:
0 - RICE-2016-ONLY.mxd
1 - STEVE RICE MAP 2021 - Shortcut.lnk
2 - STEVE RICE MAP 2021.mxd
Choose .mxd to Import into Project 0
Successfully imported Y:\PROJECTS\MNOMEN_2018\RICE-2021\RICE-2016-ONLY.mxd


In [48]:
m = getMap(aprx2)

0 - Layers
1 - Map
Choose Map Index? 0


'Active Map is: Layers'

In [ ]:
lyr = getLayer(m)

In [58]:
lyrs = listLayers(m)

0 - 2016_RICE_DATA - vis: False
1 - Wild_Ri3 - vis: True
2 - Wild_Ric - vis: True
3 - Wild_Ri4 - vis: True
4 - Wild_Ri2 - vis: True
5 - Line_gen - vis: True
6 - Wild_Ri2 - vis: True
7 - Wild_Ri3 - vis: True
8 - Wild_Ri2 - vis: True
9 - Wild_Ri2 - vis: True
10 - Wild_Ri3 - vis: True
11 - Wild_Ri4 - vis: True
12 - Wild_Ric - vis: True
13 - Wild_Ri2 - vis: True
14 - Wild_Ri3 - vis: True
15 - Wild_Ri4 - vis: True
16 - Wild_Ri2 - vis: True
17 - Wild_Ri4 - vis: True
18 - Wild_Ric - vis: True
19 - Wild_Ri3 - vis: True
20 - Wild_Ri4 - vis: True
21 - Wild_Ric - vis: True
22 - Wild_Ri2 - vis: True
23 - Wild_Ri3 - vis: True
24 - Wild_Ri4 - vis: True
25 - Wild_Ri3 - vis: True
26 - Wild_Ri4 - vis: True
27 - Point_ge - vis: True
28 - Wild_Ric - vis: True
29 - Wild_Ri3 - vis: True
30 - Wild_Ri2 - vis: True
31 - Wild_Ric - vis: True
32 - Wild_Ri4 - vis: True
33 - Wild_Ri4 - vis: True
34 - Wild_Ric - vis: True
35 - Wild_Ric - vis: True
36 - Wild_Ri2 - vis: True
37 - Wild_Ri3 - vis: True
38 - Wild_Ri4 -

In [62]:
layersFields(lyrs)

2016_RICE_DATA
Wild_Ri3
NAME: FID
NAME: Shape
NAME: GPS_Date
NAME: GPS_Time
NAME: Horz_Prec
Wild_Ric
NAME: FID
NAME: Shape
NAME: Stalks
NAME: GPS_Date
NAME: GPS_Time
NAME: Horz_Prec
Wild_Ri4
NAME: FID
NAME: Shape
NAME: GPS_Date
NAME: GPS_Time
NAME: Horz_Prec
Wild_Ri2
NAME: FID
NAME: Shape
NAME: Bed_Width
NAME: Bed_Densit
NAME: GPS_Date
NAME: GPS_Time
NAME: Horz_Prec
Line_gen
NAME: FID
NAME: Shape
NAME: Comment
NAME: GPS_Date
NAME: GPS_Time
Wild_Ri2
NAME: FID
NAME: Shape
NAME: Bed_Width
NAME: Bed_Densit
NAME: GPS_Date
NAME: GPS_Time
NAME: Horz_Prec
Wild_Ri3
NAME: FID
NAME: Shape
NAME: GPS_Date
NAME: GPS_Time
NAME: Horz_Prec
Wild_Ri2
NAME: FID
NAME: Shape
NAME: Bed_Width
NAME: Bed_Densit
NAME: GPS_Date
NAME: GPS_Time
NAME: Horz_Prec
Wild_Ri2
NAME: FID
NAME: Shape
NAME: Bed_Width
NAME: Bed_Densit
NAME: GPS_Date
NAME: GPS_Time
NAME: Horz_Prec
Wild_Ri3
NAME: FID
NAME: Shape
NAME: GPS_Date
NAME: GPS_Time
NAME: Horz_Prec
Wild_Ri4
NAME: FID
NAME: Shape
NAME: GPS_Date
NAME: GPS_Time
NAME: Horz_

In [59]:
stalks = mergeLayers(lyrs, 'Stalks', 'mergeStalks')

MATCH: Stalks - Wild_Ric - Y:\DATA_2017\RICE_DATA_2021\RICE_DATA_2016\VDRIVE\Wild_Ric
MATCH: Stalks - Wild_Ric - Y:\DATA_2017\RICE_DATA_2021\RICE_DATA_2016\RT_SIDE\Wild_Ric
MATCH: Stalks - Wild_Ric - Y:\DATA_2017\RICE_DATA_2021\RICE_DATA_2016\V_DOWN_LEFT\Wild_Ric
MATCH: Stalks - Wild_Ric - Y:\DATA_2017\RICE_DATA_2021\RICE_DATA_2016\SHORTS_BRIDGE_LEFT\Wild_Ric
MATCH: Stalks - Wild_Ric - Y:\DATA_2017\RICE_DATA_2021\RICE_DATA_2016\O_DRIVE_UP\Wild_Ric
MATCH: Stalks - Wild_Ric - Y:\DATA_2017\RICE_DATA_2021\RICE_DATA_2016\O_DRIVE_DOWN\Wild_Ric
MATCH: Stalks - Wild_Ric - Y:\DATA_2017\RICE_DATA_2021\RICE_DATA_2016\Mud_Correl_right_side\Wild_Ric
MATCH: Stalks - Wild_Ric - Y:\DATA_2017\RICE_DATA_2021\RICE_DATA_2016\MARL_DOWN_LEFT\Wild_Ric
MATCH: Stalks - Wild_Ric - Y:\DATA_2017\RICE_DATA_2021\RICE_DATA_2016\MARL_DOWN_RIGHT\Wild_Ric
Successful Merge - Output: Y:\PROJECTS\MNOMEN_2018\RICE-2021\mergeStalks-H6X1ZCHB.shp


In [56]:
def mergeLayers(lyrs, targetField, outputName):
    #try:
        mergeList, rand = [], ''.join(random.choices(string.ascii_uppercase + string.digits, k = 8))
        for lyr in lyrs:
            if lyr.isFeatureLayer == True:
                fields = arcpy.ListFields(lyr)
                for field in fields:
                    if field.name == targetField:
                        print(f'MATCH: {field.name} - {lyr.name} - {lyr.dataSource}')
                        mergeList.append(lyr)
        arcpy.Merge_management(mergeList, (f'{os.path.abspath(os.getcwd())}\\{outputName}-{rand}.shp'), '', '')
        print(f'Successful Merge - Output: {os.path.abspath(os.getcwd())}\\{outputName}-{rand}.shp')
        return mergeList
    #except:
    #    print(f'Could not merge layers for layerList: {lyrs}')

In [61]:
beds = mergeLayers(lyrs, 'Bed_Width', 'mergeBeds')

MATCH: Bed_Width - Wild_Ri2 - Y:\DATA_2017\RICE_DATA_2021\RICE_DATA_2016\VDRIVE\Wild_Ri2
MATCH: Bed_Width - Wild_Ri2 - Y:\DATA_2017\RICE_DATA_2021\RICE_DATA_2016\Mud_Correl_right_side\Wild_Ri2
MATCH: Bed_Width - Wild_Ri2 - Y:\DATA_2017\RICE_DATA_2021\RICE_DATA_2016\O_DRIVE_DOWN\Wild_Ri2
MATCH: Bed_Width - Wild_Ri2 - Y:\DATA_2017\RICE_DATA_2021\RICE_DATA_2016\RT_SIDE\Wild_Ri2
MATCH: Bed_Width - Wild_Ri2 - Y:\DATA_2017\RICE_DATA_2021\RICE_DATA_2016\SHORTS_BRIDGE_LEFT\Wild_Ri2
MATCH: Bed_Width - Wild_Ri2 - Y:\DATA_2017\RICE_DATA_2021\RICE_DATA_2016\V_DOWN_LEFT\Wild_Ri2
MATCH: Bed_Width - Wild_Ri2 - Y:\DATA_2017\RICE_DATA_2021\RICE_DATA_2016\MARL_DOWN_LEFT\Wild_Ri2
MATCH: Bed_Width - Wild_Ri2 - Y:\DATA_2017\RICE_DATA_2021\RICE_DATA_2016\O_DRIVE_UP\Wild_Ri2
MATCH: Bed_Width - Wild_Ri2 - Y:\DATA_2017\RICE_DATA_2021\RICE_DATA_2016\MARL_DOWN_RIGHT\Wild_Ri2
Successful Merge - Output: Y:\PROJECTS\MNOMEN_2018\RICE-2021\mergeBeds-OJKIZ6X4.shp


In [60]:
delMulLayers()

Current Directory: Y:\PROJECTS\MNOMEN_2018\RICE-2021
Enter Path to change Directory?<ENTER for none> 
Current Directory: Y:\PROJECTS\MNOMEN_2018\RICE-2021
Directory Contents:
5 - mergeStalks-H6X1ZCHB.shp - SHP
13 - mergeStalks-TLPNQ87G.shp - SHP
DELETE - mergeStalks-H6X1ZCHB.shp <YES/NO>YES
DELETE FEATURE CLASS - Y:\PROJECTS\MNOMEN_2018\RICE-2021\mergeStalks-H6X1ZCHB.shp ??? - YES


'File: mergeStalks-H6X1ZCHB.shp delete = true'

DELETE - mergeStalks-TLPNQ87G.shp <YES/NO>


'Y:\\PROJECTS\\MNOMEN_2018\\RICE-2021'